<a href="https://colab.research.google.com/github/RYOTA2528/data_analysis_practice/blob/main/%5BDAY21%5D%E3%83%87%E3%83%BC%E3%82%BF%E3%82%B3%E3%83%B3%E3%83%9A%E3%81%AE%E3%83%A2%E3%83%87%E3%83%AB%E6%A7%8B%E7%AF%89_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import glob #ファイルのパターン検索
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
files = glob.glob("/content/drive/MyDrive/データコンペ用/data/train/train/*.csv") #ファイルの一括読み込み
# csvファイルの読み込み
data_list = []
for file in files:
  data_list.append(pd.read_csv(file, index_col=0)) #index_col=0で0列目をインデックス
df = pd.concat(data_list)

<ipython-input-3-a1d04737cdb3>:5: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  data_list.append(pd.read_csv(file, index_col=0)) #index_col=0で0列目をインデックス
<ipython-input-3-a1d04737cdb3>:5: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  data_list.append(pd.read_csv(file, index_col=0)) #index_col=0で0列目をインデックス


In [4]:
# データクレンジングを施した処理の関数化
def data_pre(df):
  # nullを除外する
  null_list = []
  for col in df.columns:
    nonnull = df[col].count() #count()でnullは0と表示されるため
    if nonnull == 0:
      null_list.append(col)
  df = df.drop(null_list, axis = 1) # axsis=1で列から除外

  # 上記の結果、市区町村・市区町村コードが同じデータであることがわかる
  # 市区町村コードだけ残します
  df = df.drop("市区町村名", axis=1)
  # 種類も削除します
  df = df.drop("種類", axis=1)

  # "最寄駅：距離（分）" の中の文字列を一括でfloat型にかえる
  dis = {
      "30分?60分": 45,
      "1H?1H30": 75,
      "2H?": 120,
      "1H30?2H": 105
  }

  df["最寄駅：距離（分）"] = df["最寄駅：距離（分）"].replace(dis).astype(float)
  df["最寄駅：距離（分）"].value_counts()

  # 面積を変換
  df["面積（㎡）"] = df["面積（㎡）"].replace("2000㎡以上", 2000).astype(float)

  # 建築年数を西暦に変換
  y_list = {}
  for i in df["建築年"].value_counts().keys(): #keyを一つずつ取り出す
    if "平成" in i: #iの中に平成があれば
      num = float(i.split("平成")[1].split("年")[0]) #数字を取り出し格納
      year = 1988 + num #平成なので1988年に足す
    elif "令和" in i:
      num = float(i.split("令和")[1].split("年")[0])
      year = 2018 + num
    elif "昭和" in i:
      num = float(i.split("昭和")[1].split("年")[0])
      year = 1925 + num

    y_list[i] = year

  y_list["戦前"] = 1945

  y_list

  df["建築年"] = df["建築年"].replace(y_list)

  # 取引時点の〇四半期をfloat型に直す処理
  year = {
      "年第１四半期": ".25",
      "年第２四半期": ".50",
      "年第３四半期": ".75",
      "年第４四半期": ".99"
  }

  year_list = {}
  for i in df["取引時点"].value_counts().keys():
    for k, j in year.items(): #year辞書の一つ一つをとりだす
      if k in i: #iにkが入っていたら
        year_rep = i.replace(k, j)
    year_list[i] = year_rep

    year_list[i]

  df["取引時点"] = df["取引時点"].replace(year_list).astype(float)

  # 特徴量（新たなカラム）を追加する
  df["築年数"] = df["取引時点"] - df["建築年"]

  # lightbgmではobject型の読み込みができないため、cate
  # カテゴリカルデータの変換
  for col in ["種類", "都道府県名", "市区町村名", "地区名", "最寄駅：名称", "最寄駅：距離（分）",
              "間取り", "面積（㎡）", "建築年", "建物の構造", "用途", "今後の利用目的",
              "都市計画", "取引時点", "改装", "取引の事情等"]:
      df[col] = df[col].astype('category')

  return df